# Installing and Importing dependencies

In [7]:
!python --version

Python 3.10.6


In [ ]:
# !apt install swig cmake

In [4]:
# !pip install -r stable-baselines3 gymnasium[box2d] huggingface_sb3

In [5]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available. Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available. Using CPU.")

GPU is available. Using GPU: NVIDIA GeForce GTX 1660 Ti


In [8]:
# !sudo apt-get update
# !sudo apt-get install -y python3-opengl
# !apt install ffmpeg
# !apt install xvfb
# !pip install pyvirtualdisplay

In [3]:
# !pip install moviepy

In [6]:
# !pip install shimmy

In [5]:
# !pip install gym

In [ ]:
import os
os.kill(os.getpid(), 9)

: 

: 

In [1]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [2]:
import gymnasium as gym

from huggingface_sb3 import load_from_hub, package_to_hub
from huggingface_hub import notebook_login # To log to our Hugging Face account to be able to upload models to the Hub.

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

/home/darth/MLAI-projects/lunar_landing_RL/envrl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Creating and exploring Environment

In [3]:
import gymnasium as gym

# First, we create our environment called LunarLander-v2
env = gym.make("LunarLander-v2")

# Then we reset this environment
observation, info = env.reset()

for _ in range(20):
  # Take a random action
  action = env.action_space.sample()
  print("Action taken:", action)

  # Do this action in the environment and get
  # next_state, reward, terminated, truncated and info
  observation, reward, terminated, truncated, info = env.step(action)
  
  # If the game is terminated (in our case we land, crashed) or truncated (timeout)
  if terminated or truncated:
      # Reset the environment
      print("Environment is reset")
      observation, info = env.reset()

env.close()

Action taken: 3
Action taken: 1
Action taken: 3
Action taken: 3
Action taken: 0
Action taken: 1
Action taken: 2
Action taken: 3
Action taken: 1
Action taken: 1
Action taken: 2
Action taken: 1
Action taken: 0
Action taken: 2
Action taken: 1
Action taken: 2
Action taken: 0
Action taken: 0
Action taken: 3
Action taken: 2


In [4]:
env = gym.make("LunarLander-v2")
env.reset()
print("_____OBSERVATION SPACE_____ \n")
print("Observation Space Shape", env.observation_space.shape)
print("Sample observation", env.observation_space.sample()) # Get a random observation

_____OBSERVATION SPACE_____ 

Observation Space Shape (8,)
Sample observation [26.749575   31.48685    -2.089204    0.77510726  2.4600327   1.1955173
  0.5540198   0.4733921 ]


In [5]:
print("\n _____ACTION SPACE_____ \n")
print("Action Space Shape", env.action_space.n)
print("Action Space Sample", env.action_space.sample()) # Take a random action


 _____ACTION SPACE_____ 

Action Space Shape 4
Action Space Sample 3


In [6]:
# Create the vectorised environment of 16 environement
env = make_vec_env('LunarLander-v2', n_envs=16)

In [41]:
import imageio
import numpy as np

env = gym.make("LunarLander-v2", render_mode='rgb_array')
images = []
obs = env.reset()
img = env.render()
for _ in range(500):
    images.append(img)
    action = env.action_space.sample()
    obs, rewards, dones, info, truncated = env.step(action)
    img = env.render()
    # env.render()

imageio.mimsave("untrained_agent.gif", [np.array(img) for i, img in enumerate(images) if i%2 == 0], duration=40)

# Creating and training model

In [7]:
model = PPO(
    policy = 'MlpPolicy',
    env = env,
    n_steps = 1024,
    batch_size = 64,
    n_epochs = 4,
    gamma = 0.999,
    gae_lambda = 0.98,
    ent_coef = 0.01,
    verbose=1)

Using cuda device


In [8]:
# Train it for 1,000,000 timesteps
model.learn(total_timesteps=1000000)
# Save the model
model_name = "ppo-LunarLander-v0"
model.save(model_name)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 96.9     |
|    ep_rew_mean     | -197     |
| time/              |          |
|    fps             | 3536     |
|    iterations      | 1        |
|    time_elapsed    | 4        |
|    total_timesteps | 16384    |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 94.7        |
|    ep_rew_mean          | -148        |
| time/                   |             |
|    fps                  | 2526        |
|    iterations           | 2           |
|    time_elapsed         | 12          |
|    total_timesteps      | 32768       |
| train/                  |             |
|    approx_kl            | 0.005982664 |
|    clip_fraction        | 0.0221      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.38       |
|    explained_variance   | 0.00131     |
|    learning_rate        | 0.

# Evaluating trained agent

In [9]:
eval_env = Monitor(gym.make("LunarLander-v2"))
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

mean_reward=247.66 +/- 19.5818340125327


In [42]:
import imageio
import numpy as np

env = gym.make("LunarLander-v2", render_mode='rgb_array')
images = []
obs = env.reset()[0]
img = env.render()
for _ in range(500):
    images.append(img)
    action, _states = model.predict(obs)
    obs, rewards, dones, info, truncated = env.step(action)
    img = env.render()
    # env.render()

imageio.mimsave("trained_agent.gif", [np.array(img) for i, img in enumerate(images) if i%2 == 0], duration=40)

# Pushing to hub

In [45]:
# !pip install ipywidgets

In [48]:
notebook_login()
!git config --global credential.helper store

In [49]:
import gymnasium as gym

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env

from huggingface_sb3 import package_to_hub

# PLACE the variables you've just defined two cells above
# Define the name of the environment
env_id = "LunarLander-v2"

# TODO: Define the model architecture we used
model_architecture = "PPO"

## Define a repo_id
## repo_id is the id of the model repository from the Hugging Face Hub (repo_id = {organization}/{repo_name} for instance ThomasSimonini/ppo-LunarLander-v2
## CHANGE WITH YOUR REPO ID
repo_id = "darthPanda/ppo-LunarLander-v3" # Change with your repo id, you can't push with mine 😄

## Define the commit message
commit_message = "Upload PPO LunarLander-v3 trained agent"

# Create the evaluation env and set the render_mode="rgb_array"
eval_env = DummyVecEnv([lambda: gym.make(env_id, render_mode="rgb_array")])

# PLACE the package_to_hub function you've just filled here
package_to_hub(model=model, # Our trained model
               model_name=model_name, # The name of our trained model 
               model_architecture=model_architecture, # The model architecture we used: in our case PPO
               env_id=env_id, # Name of the environment
               eval_env=eval_env, # Evaluation Environment
               repo_id=repo_id, 
               is_deterministic=False,
               commit_message=commit_message)

ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.


/home/darth/MLAI-projects/lunar_landing_RL/envrl/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Saving video to /tmp/tmppcjotzmk/-step-0-to-step-1000.mp4
Moviepy - Building video /tmp/tmppcjotzmk/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmppcjotzmk/-step-0-to-step-1000.mp4



Moviepy - Done !
Moviepy - video ready /tmp/tmppcjotzmk/-step-0-to-step-1000.mp4


ffmpeg version 4.3 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/darth/anaconda3 --cc=/opt/conda/conda-bld/ffmpeg_1597178665428/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-pic --enable-pthreads --enable-shared --disable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from '/tmp/tmppcjotzmk/-step-0-to-step-1000.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compa

ℹ Pushing repo darthPanda/ppo-LunarLander-v3 to the Hugging Face
Hub


policy.pth:   0%|          | 0.00/43.3k [00:00<?, ?B/s]





policy.pth:  38%|███▊      | 16.4k/43.3k [00:01<00:01, 15.8kB/s]






pytorch_variables.pth: 100%|██████████| 431/431 [00:01<00:00, 320B/s]
policy.pth: 100%|██████████| 43.3k/43.3k [00:01<00:00, 24.3kB/s]
policy.optimizer.pth: 100%|██████████| 87.9k/87.9k [00:01<00:00, 49.1kB/s]
ppo-LunarLander-v0.zip: 100%|██████████| 147k/147k [00:02<00:00, 69.9kB/s]




Upload 4 LFS files: 100%|██████████| 4/4 [00:02<00:00,  1.51it/s]


ℹ Your model is pushed to the Hub. You can view your model here:
https://huggingface.co/darthPanda/ppo-LunarLander-v3/tree/main/


'https://huggingface.co/darthPanda/ppo-LunarLander-v3/tree/main/'

# Loading from hub

In [50]:
from huggingface_sb3 import load_from_hub
repo_id = "darthPanda/ppo-LunarLander-v3" # The repo_id
filename = "ppo-LunarLander-v0.zip" # The model filename.zip

# When the model was trained on Python 3.8 the pickle protocol is 5
# But Python 3.6, 3.7 use protocol 4
# In order to get compatibility we need to:
# 1. Install pickle5 (we done it at the beginning of the colab)
# 2. Create a custom empty object we pass as parameter to PPO.load()
custom_objects = {
            "learning_rate": 0.0,
            "lr_schedule": lambda _: 0.0,
            "clip_range": lambda _: 0.0,
}

checkpoint = load_from_hub(repo_id, filename)
model = PPO.load(checkpoint, custom_objects=custom_objects, print_system_info=True)

== CURRENT SYSTEM INFO ==
- OS: Linux-5.19.0-46-generic-x86_64-with-glibc2.35 # 47~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Wed Jun 21 15:35:31 UTC 2
- Python: 3.10.6
- Stable-Baselines3: 2.0.0
- PyTorch: 2.0.1+cu117
- GPU Enabled: True
- Numpy: 1.25.1
- Cloudpickle: 2.2.1
- Gymnasium: 0.28.1
- OpenAI Gym: 0.26.2

== SAVED MODEL SYSTEM INFO ==
- OS: Linux-5.19.0-46-generic-x86_64-with-glibc2.35 # 47~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Wed Jun 21 15:35:31 UTC 2
- Python: 3.10.6
- Stable-Baselines3: 2.0.0
- PyTorch: 2.0.1+cu117
- GPU Enabled: True
- Numpy: 1.25.1
- Cloudpickle: 2.2.1
- Gymnasium: 0.28.1
- OpenAI Gym: 0.26.2



In [51]:
model